In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import bokeh as bk

In [2]:
### Import data

data_dir = '/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs'
filename = 'Vahid_std.csv'
filepath = os.path.join(data_dir,filename)

df = pd.read_csv(filepath)
print(df)

       Machine_Displacement  Machine_N_cycles  DIC_index  DIC_N_cycles  \
0                  0.001037               1.0        NaN           NaN   
1                  0.001262               1.0        NaN           NaN   
2                  0.001392               1.0        NaN           NaN   
3                  0.001506               1.0        NaN           NaN   
4                  0.001541               1.0        NaN           NaN   
...                     ...               ...        ...           ...   
29895              0.009387         1197000.0        NaN           NaN   
29896              0.009945         1197000.0        NaN           NaN   
29897              0.010576         1197000.0        NaN           NaN   
29898              0.011272         1197000.0        NaN           NaN   
29899              0.011516         1197000.0        NaN           NaN   

       DIC_exy  DIC_exx  DIC_crack_length  Th_time  DIC_eyy  Th_specimen_max  \
0          NaN      NaN        